In [1]:
import dotenv
import os
dotenv.load_dotenv()

True

In [2]:
OPENAI_API=os.getenv("OPENAI_KEY")

In [3]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0.1,openai_api_key=OPENAI_API,model_name="gpt-3.5-turbo-1106")


In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory

In [5]:
prompt=ChatPromptTemplate.from_messages([
    ('system',"Your name is SkauraSky. You are in customer serivice who comminicates information about the flights that are there."),
    ('human',"{Question}")

])
chain=prompt | llm 

In [6]:
import pandas as pd
from langchain_experimental.tools.python.tool import PythonREPL
path=os.path.dirname(os.getcwd())
path=os.path.join(path,'Datasets/Data_Train_Clean.xlsx')
df=pd.read_excel(path)
python=PythonREPL(locals={'df':df})



In [7]:
df


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0.0,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2.0,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2.0,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1.0,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1.0,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,0.0,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,0.0,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,0.0,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,0.0,No info,12648


In [11]:
from langchain.agents import Tool, AgentType, initialize_agent
from langchain_experimental.agents import create_pandas_dataframe_agent
tools=[
    Tool(
        name="Give flight Details",
        func=python.run,
        description="useful for when you need to answer questions about flights"
    )
]

In [12]:
agent = initialize_agent(tools, 
                                 llm, 
                                 agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                                 verbose=True
                                 )
        # define q and a function for frontend
def get_response(user_input):
            response = agent.run(user_input)
            return response

In [13]:
agent.invoke("What flight are there from Banglore to  Delhi in the range from 1st march to 3rd of march")



> Entering new AgentExecutor chain...


PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_iEsOaUGlUaXl8pwxVlju7fbu` does not have access to model `gpt-3.5-turbo-1106`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [22]:
from langchain_community.document_loaders.dataframe import DataFrameLoader

document=DataFrameLoader(df)




In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain.memory import ChatMessageHistory
llm=ChatOpenAI(temperature=0.1,openai_api_key=OPENAI_API,model_name="gpt-3.5-turbo-1106")
chat_history = ChatMessageHistory()

In [32]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



prompt = ChatPromptTemplate.from_messages([
    ('system', "Your name is SkauraSky. You are in customer service who communicates information about the flights that are there."),
    ('system', "{context}"),  # Add a new line for context placeholder
    MessagesPlaceholder(variable_name=""),
])

chain = create_stuff_documents_chain(llm, prompt)


In [33]:
from typing import Dict

from chromadb.config import C
from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input,
).assign(
    answer=chain,
)

In [37]:
retrieval_chain_with_only_answer = (
    RunnablePassthrough.assign(
        context=parse_retriever_input,
    )
    |chain
)

retrieval_chain_with_only_answer.invoke(
    {
        "messages":chat_history.messages,
    },
)

IndexError: list index out of range